In [1]:
import pandas as pd
import numpy as np

In [2]:
# Pede a matrícula e recupera as informações de X e Y que serão usadas para a resolução do problema
matricula = input('Qual a sua matrícula?')
DIG_X = int(matricula[-3:][0:1])
DIG_Y = int(matricula[-1:])

Qual a sua matrícula? 12.112.451-5


In [3]:
# Cria as tabelas fornecidas pelo enunciado
df = {
    'NUM_MAQ_QUE_HR': [1,2,3,4,5,6,7,8],
    'NUM_OCOR': [22,42,94,124,82,36,20,4]
}
df = pd.DataFrame(df, index = range(1,9))


df_X = {
    'DIG_X': [0,1,2,3,4,5,6,7,8,9],
    '$_HMP_QUEBRA': [22.2,21.6,20.2,22.4,21.8,20.5,22.6,21.2,20.8,20.4]
}
df_X = pd.DataFrame(df_X)


df_Y = {
    'DIG_Y': [0,1,2,3,4,5,6,7,8,9],
    '$_HMP_RESER': [12,14,16,18,11,13,15,17,19,10]
}
df_Y = pd.DataFrame(df_Y)

In [59]:
# Cria colunas e recupera informações da tabelas que serão usadas na solução

df['FREQ'] = df['NUM_OCOR']/df['NUM_OCOR'].sum()
C_HMP_QUEBRA = df_X[df_X['DIG_X'] == DIG_X].iloc[0,1]
C_HMP_RESER = df_Y[df_Y['DIG_Y'] == DIG_Y].iloc[0,1]

In [5]:
# Cria a matrix já com a relação de subtração entre os índices fornecidos de 1 a 8 para relação de máquina reserva e máquina quebrada
v = np.array(df['NUM_MAQ_QUE_HR'])

arr = np.array(v[:, None] - v)

df_matrix = pd.DataFrame(arr, index = [1,2,3,4,5,6,7,8] ,columns = [1,2,3,4,5,6,7,8])

df_matrix

,1,2,3,4,5,6,7,8
1,0,-1,-2,-3,-4,-5,-6,-7
2,1,0,-1,-2,-3,-4,-5,-6
3,2,1,0,-1,-2,-3,-4,-5
4,3,2,1,0,-1,-2,-3,-4
5,4,3,2,1,0,-1,-2,-3
6,5,4,3,2,1,0,-1,-2
7,6,5,4,3,2,1,0,-1
8,7,6,5,4,3,2,1,0


In [29]:
# Relaciona o custo de quebra, custo de maquina reserva e multiplica pela frequência da tabela fornecida pelo enunciado.
## Cria-se a coluna com a somatória de custo total também

df_custos = df_matrix.copy()
for col in df_custos.columns:
    df_custos[col] = list(map(lambda x: abs(x * C_HMP_RESER) if x > 0 else abs(x * C_HMP_QUEBRA), df_custos[col]))
    df_custos[col] = df['FREQ'].iloc[int(col)-1] * df_custos[col]
    
df_custos['CUSTO_TOTAL'] = df_custos.sum(axis = 1, skipna = True)

df_custos

,1,2,3,4,5,6,7,8,CUSTO_TOTAL
1,0.000000,2.139623,9.577358,18.950943,16.709434,9.169811,6.113208,1.426415,64.086792
2,0.674528,0.000000,4.788679,12.633962,12.532075,7.335849,5.094340,1.222642,44.282075
3,1.349057,1.287736,0.000000,6.316981,8.354717,5.501887,4.075472,1.018868,27.904717
4,2.023585,2.575472,2.882075,0.000000,4.177358,3.667925,3.056604,0.815094,19.198113
5,2.698113,3.863208,5.764151,3.801887,0.000000,1.833962,2.037736,0.611321,20.610377
6,3.372642,5.150943,8.646226,7.603774,2.514151,0.000000,1.018868,0.407547,28.714151
7,4.047170,6.438679,11.528302,11.405660,5.028302,1.103774,0.000000,0.203774,39.755660
8,4.721698,7.726415,14.410377,15.207547,7.542453,2.207547,0.613208,0.000000,52.429245


In [66]:
# Gera a resposta do problema

C_Min = df_custos['CUSTO_TOTAL'].min()

N_Maquinas_Reservas = df_custos.index[df_custos['CUSTO_TOTAL'] == df_custos['CUSTO_TOTAL'].min()].tolist()[0]
N_Maquinas_Reservas_Final = N_Maquinas_Reservas

for a in range(N_Maquinas_Reservas + 1, df_custos['CUSTO_TOTAL'].count()+1):
    
    if df_custos['CUSTO_TOTAL'][a] - C_Min < C_HMP_RESER:
        N_Maquinas_Reservas_Final = a
    else:
        N_Maquinas_Reservas_Final = N_Maquinas_Reservas_Final
        
if N_Maquinas_Reservas == N_Maquinas_Reservas_Final:
    print('Serão alocadas ' +
          str(N_Maquinas_Reservas_Final) +
          ' maquinas reservas com o custo total de $' +
          "{:.2f}".format(round(df_custos['CUSTO_TOTAL'][N_Maquinas_Reservas_Final],2)).replace('.',',') +
          ' com a cobertura das ocorrências de ' +
          "{:.2f}".format(round(df['FREQ'][df['NUM_MAQ_QUE_HR'] <= N_Maquinas_Reservas_Final].sum()*100,2)).replace('.',',') +
          '% dos casos. Não houve a necessidade de alocar mais máquinas, pois o custo de alocação de mais uma máquina foi maior do que o custo da máquina reserva[$' +
          "{:.2f}".format(round(float(C_HMP_RESER),2)).replace('.',',') +
          '].')
else:
    print('Serão alocadas ' +
          str(N_Maquinas_Reservas_Final) +
          ' maquinas reservas com o custo total de $' +
          "{:.2f}".format(round(df_custos['CUSTO_TOTAL'][N_Maquinas_Reservas_Final],2)).replace('.',',') +
          ' com a cobertura das ocorrências de ' +
          str(round(df['FREQ'][df['NUM_MAQ_QUE_HR'] <= N_Maquinas_Reservas_Final].sum()*100,2)).replace('.',',') +
          '% dos casos. Foram alocadas ' +
          str(N_Maquinas_Reservas_Final-N_Maquinas_Reservas) +
          ' maquinas a mais em relação ao custo mínimo da tabela, isto foi feito devido a diferença de custo de alocação dessas máquinas [$' +
          "{:.2f}".format(round(df_custos['CUSTO_TOTAL'][N_Maquinas_Reservas_Final] - df_custos['CUSTO_TOTAL'][N_Maquinas_Reservas],2)).replace('.',',') +
          '] em relação ao custo mínimo ser menor do que o custo de uma máquina reserva [$' +
          "{:.2f}".format(round(float(C_HMP_RESER),2)).replace('.',',') +
          '].')

Serão alocadas 6 maquinas reservas com o custo total de $28,71 com a cobertura das ocorrências de 94,34% dos casos. Foram alocadas 2 maquinas a mais em relação ao custo mínimo da tabela, isto foi feito devido a diferença de custo de alocação dessas máquinas [$9,52] em relação ao custo mínimo ser menor do que o custo de uma máquina reserva [$13,00].


In [64]:
N_Maquinas_Reservas_Final

4

In [58]:
df['NUM_OCOR'].mean()

53.0